In [39]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
all_data = pd.read_csv("crx.data", header = None)
all_data.replace('?', np.NaN, inplace=True)

data_train, data_test, labels_train, labels_test = train_test_split(all_data.iloc[:,:15], all_data.iloc[:,[15]], test_size=0.8, random_state=78)

imp = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
cat_data_train = list ()
num_data_train = list ()
cat_data_test = list ()
num_data_test = list ()
cat_cols = [1,4,5,6,7,9,10,12,13]
for col in cat_cols :
  cat_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  cat_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))
num_cols = [2,3,8,11,14,15]
imp = SimpleImputer(missing_values=np.NaN, strategy="mean")
for col in num_cols :
  num_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  num_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))

j1 , j2 = 0, 0

idata_train = pd.DataFrame()
idata_test = pd.DataFrame()
for col in range(1,16) :
  if j2 < 9 and cat_cols[j2] == col :
    idata_train = pd.concat([idata_train,cat_data_train[j2]],join="outer",axis=1)
    idata_test = pd.concat([idata_test, cat_data_test[j2]],join="outer",axis=1)
    j2 = j2 + 1
  else :
    idata_train = pd.concat([idata_train,num_data_train[j1]],join="outer",axis=1)
    idata_test = pd.concat([idata_test,num_data_test[j1]],join="outer",axis=1)    
    j1 = j1 + 1

idata_train.columns = list(range(1,16))
idata_test.columns = list(range(1,16))

# NEW CODE HERE -- NOTE : columns must be 46 and that is what the output shows
temp = pd.concat([idata_train,idata_test], join="outer", axis=0)
dummy_data = pd.get_dummies(temp, [0,3,4,5,6,8,9,11,12])
np_dummy_data = dummy_data.apply(pd.to_numeric).values

np_data_train = np_dummy_data[:138,:] #οι γραμμές δεν αλλάζουν θέση οπότε αφού σειριακά
np_data_test = np_dummy_data[138:,:]  #ενώσαμε τα train, test σειριακά θα τα χωρίσουμε

mapping = {'-' : 0, '+' : 1}
labels_train[15] = labels_train[15].map(mapping)
labels_test[15] = labels_test[15].map(mapping)

np_labels_train = labels_train.values.flatten()
np_labels_test = labels_test.values.flatten()

In [40]:
print(np_data_train, np_data_train.shape)

[[29.5    0.46   0.54  ...  1.     0.     0.   ]
 [36.75   5.125  5.    ...  1.     0.     0.   ]
 [31.33  19.5    7.    ...  1.     0.     0.   ]
 ...
 [54.83  15.5    0.    ...  1.     0.     0.   ]
 [34.17   9.17   4.5   ...  1.     0.     0.   ]
 [21.92  11.665  0.085 ...  1.     0.     0.   ]] (138, 46)


In [41]:
print(np_data_test, np_data_test.shape)

[[27.75        1.29        0.25       ...  0.          0.
   1.        ]
 [21.08       10.085       1.25       ...  1.          0.
   0.        ]
 [23.5         1.5         0.875      ...  1.          0.
   0.        ]
 ...
 [32.86377778  1.5         0.         ...  1.          0.
   0.        ]
 [25.25       13.5         2.         ...  1.          0.
   0.        ]
 [20.25        9.96        0.         ...  1.          0.
   0.        ]] (552, 46)


In [42]:
print(np_labels_train, np_labels_train.shape)

[1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1
 1 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0
 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0
 1 0 0 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0] (138,)


In [43]:
print(np_labels_test, np_labels_test.shape)

[0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 1 0 0
 0 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0
 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1 0
 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0
 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1
 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 1
 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 1 1 0
 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 1 1 